In [28]:
#hide
#default_exp tools.utils
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# CLI utils

> Utils used in cli tools.

In [29]:
#export
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import joblib

from hpsearch.config.hpconfig import get_experiment_manager
import hpsearch.config.hp_defaults as dflt

In [30]:
#for tests
import pytest
import os

from dsblocks.utils.utils import remove_previous_results

from hpsearch.examples.dummy_experiment_manager import generate_data

## include best and last experiment

In [31]:
#export
def include_best_and_last_experiment (metrics, experiments=[-1, -2], run_number=0, 
                                      op='max', manager_path=dflt.manager_path, em=None):
    if em is None: em = get_experiment_manager (manager_path=manager_path)
    path_experiments = em.path_experiments
    df = em.get_experiment_data ()
    remove_experiment = None
    for i in range(len(experiments)):
        if experiments[i] == -1:
            experiment_number = joblib.load(path_experiments/'current_experiment_number.pkl')
            experiments[i] = experiment_number

        if experiments[i] == -2:
            first_metric = metrics[0]
            if len(metrics)>1:
                print (f'we use the first metric {first_metric} in given list {metrics} '
                        'for obtaining the best experiment')
            
            score_column = (dflt.scores_col, first_metric, run_number)
            if score_column in df.columns:
                if op=='max':
                    experiments[i] = df[score_column].astype(float).idxmax()
                else:
                    experiments[i] = df[score_column].astype(float).idxmin()
            else:
                if remove_experiment is not None: raise ValueError ('more than one experiment is -2')
                remove_experiment = i
    if remove_experiment is not None: del experiments[remove_experiment]
    return experiments

## set_metric

In [32]:
#export
def set_metric (em, metric=None):
    assert metric is not None or em.key_score is not None
    if metric is None: 
        metric = em.key_score
        metric = [metric]
    else:
        if not isinstance (metric, list): metric = [metric]
        em.key_score = metric[0]
    if em.key_score is None: em.key_score = metric[0]
    return metric